In [1]:
import pandas as pd
import numpy as np
import csv
import datetime 
# import time

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from src import settings

recobell_root_path = settings.DATA_ROOT_PATH + "/site_data/recobell/"

In [3]:
explicit_recobell_df = pd.read_csv(recobell_root_path + '_implicit.clean.txt', header=None, names=['uid', 'itemid', 'time'])
print(explicit_recobell_df.head())
# active_item_df = pd.read_csv(recobell_root_path + 'item_repr.txt', header=None, names=['itemid', 'repr'], delimiter=',', quotechar='|', skiprows=1)
# active_item_df.head()

       uid   itemid        time
0  b15f9e5  d9886b1  1471066727
1  8007201  e295b87  1471156894
2  8007201  cedc044  1471156677
3  99055ba  4f3ecec  1470917081
4  99055ba  ee3f827  1470916631


In [4]:
# load list key for i2index
with open(recobell_root_path + 'i2index.txt') as f:
    active_item_dict = {str(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_item_list = active_item_dict.keys()
print(len(active_item_list))

118293


In [5]:
# list key index from u2index.
with open(recobell_root_path + 'u2index.txt') as f:
    active_user_dict = {str(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_user_list = active_user_dict.keys()
print(len(active_user_list))

206203


In [6]:
# loai bo tuong tac ma user khong co trong active_user_list
explicit_recobell_df = explicit_recobell_df[explicit_recobell_df.uid.isin(active_user_list)]
explicit_recobell_df = explicit_recobell_df[explicit_recobell_df.itemid.isin(active_item_list)]

print 'explicit recobell df before:'
print (explicit_recobell_df.head())


explicit recobell df before:
       uid   itemid        time
3  99055ba  4f3ecec  1470917081
4  99055ba  ee3f827  1470916631
5  99055ba  4f958e5  1470844977
6  99055ba  51f86bb  1470811668
7  99055ba  32f8032  1470811985


In [7]:
print 'Sorting by uid and time giam dan (cang xa so voi hien tai):'
# a = retail_rocket_df.sort_values(by=['uid'])
explicit_recobell_df = explicit_recobell_df.groupby(["uid"]).apply(lambda x: x.sort_values(["time"], ascending = False)).reset_index(drop=True)
print (explicit_recobell_df[:10])

Sorting by uid and time giam dan (cang xa so voi hien tai):
       uid   itemid        time
0  0000516  2d5ff79  1470970249
1  0000516  f5668cf  1470964138
2  0000516  ad0a963  1470964065
3  0000516  2d5ff79  1470963920
4  0000516  2d5ff79  1470962566
5  0000516  2d5ff79  1470962554
6  0000516  2d5ff79  1470962463
7  0000516  2d5ff79  1470962202
8  0000516  2d5ff79  1470962090
9  0000516  2d5ff79  1470961539


In [ ]:
# thay the rating ban dau boi user_id, item_id moi trong u2index va i2index.
explicit_recobell_df = explicit_recobell_df.apply(lambda x: [active_user_dict[x[0]], active_item_dict[x[1]]], axis=1, result_type='expand')
print(explicit_recobell_df.head())

In [11]:
explicit_recobell_df.to_csv(recobell_root_path + 'ui_ex.txt', index=False, header=False)

# Sorting ratings.txt

In [2]:
recobell_df = pd.read_csv(recobell_root_path + 'ratings.txt', header=None, names=['uid', 'itemid', 'time_implicit','number_im','time_explicit','number_ex'])
print(recobell_df.head())

   uid  itemid  time_implicit  number_im  time_explicit  number_ex
0    0       0     1470917081          2             -1          0
1    0       1     1470916631          1             -1          0
2    0       2     1470844977          1             -1          0
3    0       3     1470811668          1             -1          0
4    0       4     1470811985          2             -1          0


In [3]:
print 'Sorting by uid and time tang dan (tu qua khu cho den bay gio):'
recobell_df = recobell_df.groupby(["uid"]).apply(lambda x: x.sort_values(["time_implicit"], ascending = True)).reset_index(drop=True)
print (recobell_df[:50])

Sorting by uid and time tang dan (tu qua khu cho den bay gio):
    uid  itemid  time_implicit  number_im  time_explicit  number_ex
0     0      15     1470701505          1             -1          0
1     0       5     1470701744          2             -1          0
2     0       3     1470811668          1             -1          0
3     0      14     1470811765          1             -1          0
4     0       4     1470811985          2             -1          0
5     0       2     1470844977          1             -1          0
6     0      13     1470845413          1             -1          0
7     0       1     1470916631          1             -1          0
8     0      12     1470917017          1             -1          0
9     0       0     1470917081          2             -1          0
10    0    1182     1471189847          1             -1          0
11    0     277     1471190697          2             -1          0
12    1      10     1470644889          3            

In [4]:
# in ra file 
recobell_df.to_csv(recobell_root_path + 'ratings_sort_tuc.txt', index=False, header=False)

# generate item representation.

In [9]:
# using item_repr.txt was made before to generate vector for this item.
active_item_df = pd.read_csv(recobell_root_path + 'item_repr.txt', header=None, names=['itemid', 'repr'], delimiter=',', quotechar='|', skiprows=1)
print(active_item_df.head())

   itemid                                               repr
0    3506  [[17, 0.25], [80, 0.25], [219, 0.25], [464, 0....
1    2293  [[16, 0.25], [46, 0.25], [335, 0.25], [1892, 0...
2    6555  [[9, 0.25], [83, 0.25], [153, 0.25], [1854, 0....
3    8448  [[6, 0.25], [82, 0.25], [249, 0.25], [832, 0.25]]
4    1818  [[1, 0.25], [38, 0.25], [286, 0.25], [841, 0.25]]
/home/tucng/Desktop/RS/project/src/site_data/recobell/


In [15]:
# make a dictionary for each item 
# each of element is a key of item and value of categories
item_repr_dict = {}
for _, row in active_item_df.iterrows():
    item_repr_dict[row[0]] = row[1]

print(item_repr_dict[0])
print(item_repr_dict[3506])

[[14, 0.25], [73, 0.25], [426, 0.25], [1620, 0.25]]
[[17, 0.25], [80, 0.25], [219, 0.25], [464, 0.25]]


In [17]:
explicit_recobell_df['num'] = 1

In [19]:
explicit_recobell_df.columns = ['uid', 'itemid', 'num']
explicit_recobell_df.head()

,uid,itemid,num
0,0,0,1
1,0,3,1
2,0,4,1
3,0,6,1
4,0,7,1


In [22]:
import json

def json_string_to_dense_vector(string_vector, dimensions):
    """
    Chuyen vector thua dang string: [[9, 0.010176822], [118, 0.010578092], [264, 0.020403702]]
    ve vector thuong K chieu cua numpy
    """
    K = dimensions

    vector = np.zeros(K, dtype=np.float32)
    if string_vector is None or len(string_vector) < 1:
        return vector

    x = json.loads(string_vector)
    
    for element in x:
        vector[element[0]] = element[1]
    return vector

In [23]:
def dense_vector_to_list_sparse_vector(vector):
    """
    Chuyen vector np ve vector thua dang string: [[9, 0.010176822], [118, 0.010578092], [264, 0.020403702]]
    """
    K = len(vector)
    threshold = 1e-6

    list_temp = []
    for i in range(0, K):
        if vector[i] > threshold:
            list_temp.append([i, float(vector[i])])
    return list_temp

In [24]:
def list_sparse_vector_to_json_string(sv):
    # type: (list) -> str
    """Convert vector in the form list of (int, float) to string

    Parameters
    ----------
    sv : list of (int, float)
        sparse_vector to be converted

    Examples
    ---------
    >>> sparse_vector = [(1, 0.123232), (2, 5.34234234)]
    >>> list_sparse_vector_to_json_string(sparse_vector)
    '[[1, 0.123232], [2, 5.34234234]]'
    """

    sum = 0
    list_sparse = []

    for item in sv:
        sum += item[1]

    for item in sv:
        list_sparse.append([int(item[0]), float(item[1] / sum)])

    return json.dumps(list_sparse)


In [ ]:
def add_avg()

In [27]:
explicit_recobell_df.head()

,uid,itemid,num
0,0,0,1
1,0,3,1
2,0,4,1
3,0,6,1
4,0,7,1


In [25]:
# apply into each row of explicit_recobell_df
t = explicit_recobell_df.apply(lambda x: [x[0], json_string_to_dense_vector([x[1]], 1939), x[2]], axis=1, result_type='expand').groupby(by=['0'])
print(t.head())

TypeError: ('expected string or buffer', u'occurred at index 0')

In [22]:
# item_recobell_df = pd.read_csv(recobell_root_path + 'raw_data/site_product.csv000', header=None, names=['itemid', 'price', 'cat1', 'cat2', 'cat3', 'cat4', 'brandid'])
# view_recobell_df = pd.read_csv(recobell_root_path + 'raw_data/tiny_site_view_log.csv000', header=None, names=['server_time', 'device', 'session_id', 'uid', 'itemid'])
order_recobell_df = pd.read_csv(recobell_root_path + 'raw_data/tiny_site_order_log.csv000', header=None, names=['server_time', 'device', 'session_id', 'uid', 'itemid', 'order_id', 'quantity'])
view_recobell_df.head()
# preprocessing with recobell datasets

,server_time,device,session_id,uid,itemid
0,2016-08-13 12:38:47.512,MA,bgF9eb8ynC,b15f9e5,d9886b1
1,2016-08-14 13:41:34.30,MA,yzbfOf5b4e,8007201,e295b87
2,2016-08-14 13:37:57.682,MA,yzbfOf5b4e,8007201,cedc044
3,2016-08-11 19:04:41.939,MA,YBpaGQxzg3,99055ba,4f3ecec
4,2016-08-11 18:57:11.63,MA,YBpaGQxzg3,99055ba,ee3f827


In [3]:
item_recobell_df.nunique()

itemid     422516
price       24607
cat1           18
cat2           80
cat3          342
cat4         1499
brandid     10584
dtype: int64

In [18]:
active_item_df = pd.read_csv(recobell_root_path + 'item_repr.txt', header=None, names=['itemid', 'repr'], delimiter=',', quotechar='|')
active_item_df.head()

,itemid,repr
0,5919d61,"[[17, 0.25], [80, 0.25], [219, 0.25], [464, 0...."
1,1073c34,"[[16, 0.25], [46, 0.25], [335, 0.25], [1892, 0..."
2,559fc56,"[[9, 0.25], [83, 0.25], [153, 0.25], [1854, 0...."
3,67da1f9,"[[6, 0.25], [82, 0.25], [249, 0.25], [832, 0.25]]"
4,b5eaeaa,"[[1, 0.25], [38, 0.25], [286, 0.25], [841, 0.25]]"


In [33]:
z = active_item_df[active_item_df.itemid.isin(active_item_list)]
z = z.drop_duplicates(['itemid'])
z.count()

itemid    118293
repr      118293
dtype: int64

In [34]:
t = z.apply(lambda x: [active_item_dict[x[0]], x[1]], axis=1, result_type='expand')
t.head()

,0,1
0,3506,"[[17, 0.25], [80, 0.25], [219, 0.25], [464, 0...."
1,2293,"[[16, 0.25], [46, 0.25], [335, 0.25], [1892, 0..."
2,6555,"[[9, 0.25], [83, 0.25], [153, 0.25], [1854, 0...."
3,8448,"[[6, 0.25], [82, 0.25], [249, 0.25], [832, 0.25]]"
4,1818,"[[1, 0.25], [38, 0.25], [286, 0.25], [841, 0.25]]"


In [37]:
import csv
t.to_csv(recobell_root_path + 'item_repr.txt', index=False, header=False, quotechar='|', quoting=csv.QUOTE_ALL)

In [28]:
y = active_item_df.loc[:, 'itemid'].drop_duplicates()
y.count()

422516

In [29]:
x = y.tolist()
print(len(x))

422516


In [30]:
new_view_df = view_recobell_df[view_recobell_df.itemid.isin(x)]
new_order_df = order_recobell_df[order_recobell_df.itemid.isin(x)]
new_view_df.count()

server_time    4521505
device         4521505
session_id     4521505
uid            4521505
itemid         4521505
dtype: int64

In [31]:
new_view_df.to_csv(recobell_root_path + 'raw_data/new_tiny_site_view_log.csv000', index=False, header=False)
new_order_df.to_csv(recobell_root_path + 'raw_data/new_tiny_site_order_log.csv000', index=False, header=False)

In [27]:
z.head()

,itemid,repr
0,5919d61,"[[17, 0.25], [80, 0.25], [219, 0.25], [464, 0...."
1,1073c34,"[[16, 0.25], [46, 0.25], [335, 0.25], [1892, 0..."
2,559fc56,"[[9, 0.25], [83, 0.25], [153, 0.25], [1854, 0...."
3,67da1f9,"[[6, 0.25], [82, 0.25], [249, 0.25], [832, 0.25]]"
4,b5eaeaa,"[[1, 0.25], [38, 0.25], [286, 0.25], [841, 0.25]]"


In [ ]:
z.to_csv(recobell_root_path + 'item_repr.txt', index=False, header=False)

In [61]:
a =  item_recobell_df.loc[:, ['cat2', 'cat3']]
x =  item_recobell_df.loc[:, ['cat2']]

In [63]:
y = a.drop_duplicates().groupby(['cat3']).count()
y.loc[y.cat2 > 1]

,cat2
cat3,
0ea22c8,2
142a302,2
23bebb0,2
3919b77,2
44650a9,7
512f480,2
58961c3,2
5b889c3,2
7efeb2d,3


In [59]:
item_recobell_df[item_recobell_df.cat2 == 'bd0b0a5'].drop_duplicates(['cat1', 'cat2'])

,itemid,price,cat1,cat2,cat3,cat4,brandid
9,eb32384,67410,d9d0ba9,bd0b0a5,f1c99bd,af4f9f8,7a9f4fc
114195,257677b,39000,66dd73d,bd0b0a5,58961c3,a457db3,fd23080


In [38]:
b = set(item_recobell_df.cat1) | set(item_recobell_df.cat2) | set(item_recobell_df.cat3) | set(item_recobell_df.cat4)

In [39]:
len(b)

1838

In [10]:
c = list(set(item_recobell_df.cat1))

In [1]:
len(c)
c

NameError: name 'c' is not defined

# Pre-process for movielens data

In [3]:
ratings.head()

,uid,itemid,rating,server_time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
count_rating = ratings.groupby(['uid'])[['rating']].count()
count_rating[count_rating['rating'] < 20]

,rating
uid,


In [5]:
mean_rating = ratings.groupby(['uid']).agg({'rating': 'mean'})
mean_rating['uid'] = mean_rating.index
mean_rating[mean_rating['rating'] < 2]

# ratings[ratings['uid'] == 2744]

,rating,uid
uid,,
2744,1.304348,2744
3598,1.015385,3598
4349,1.962963,4349
4486,1.058824,4486
4539,1.815126,4539
5334,1.927273,5334
5850,1.844828,5850


In [7]:
count_rating['rating'].max()

2314

In [8]:
count_rating['rating'].min()

20

In [48]:
mean_rating
with open(ml_1m_root_path + 'u2index.txt') as f:
    active_user_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_user_list = active_user_dict.keys()
print(len(active_user_list))
new_mean_rating = mean_rating.apply(lambda x: pd.Series([active_user_dict[x['uid']], x['rating']], index=['uid', 'rating']), axis=1)

6040


In [55]:
new_mean_rating['uid'] = new_mean_rating['uid'].apply(lambda x: int(x))

In [6]:
mean_rating.to_csv(ml_1m_root_path + 'user_mean_rating.txt', header=False, index=False, quotechar='|', quoting=csv.QUOTE_ALL)